In [21]:
open Core

In [22]:
1 :: (2 :: (3 :: []))

- : int list = [1; 2; 3]


In [23]:
1 :: 2 :: 3:: [];;

- : int list = [1; 2; 3]


可以看到，`::`是右结合的，这说明我们可以不用右括号来建立列表。空表`[]`用来结束一个列表。需要说明，空表是多态的，这意味者它可以用于任意类型的元素，如下所示

In [24]:
let empty = [];;

val empty : 'a list = []


In [25]:
3::empty;;

- : int list = [3]


In [26]:
"three"::empty;;

- : string list = ["three"]


Ocaml列表实际上是单列表

从一个列表中过滤掉列表中等于特定值的元素

In [27]:
let rec drop_value l to_drop = 
match l with
| [] -> []
| hd :: tl ->
let new_tl = drop_value tl to_drop in
if hd = to_drop then new_tl else hd :: new_tl

val drop_value : 'a list -> 'a -> 'a list = <fun>


In [28]:
drop_value [1;2;3;2] 2;;

- : int list = [1; 3]


一般的，模式匹配要比你手工编写的其他代码更加高效

In [29]:
let s = "." ^ "." ^ "." ^  "." ^ "." ^ "." ^".";

val s : string = "......."


将会分配长度为2,3,4,5,6和7字符串

In [33]:
let s = String.concat [".";".";".";".";".";"."]

val s : Core.String.t = "......"


只分配一个长度为7的字符串。大规模的组装字符串的时，"^"会带来严重问题

计算列的最大宽度

In [31]:
let max_widths header rows = 
let lengths l = List.map ~f:String.length l in
List.fold rows
~init:(lengths header)
~f:(fun acc row -> List.map2_exn ~f:Int.max acc (lengths row))

val max_widths :
  Core.String.t Core.List.t ->
  Core.String.t Core.List.t Core.List.t -> Core.Int.t Core.List.t = <fun>


In [34]:
List.map2_exn

- : 'a Core.List.t -> 'b Core.List.t -> f:('a -> 'b -> 'c) -> 'c Core.List.t
= <fun>


In [36]:
let render_separator widths = 
let pieces = List.map widths
~f:(fun w -> String .make (w + 2) '-') in
"|" ^ String.concat ~sep:"+" pieces ^ "|"

val render_separator : int Core.List.t -> string = <fun>


In [37]:
render_separator [3;6;2];

- : string = "|-----+--------+----|"


In [47]:
let rec lengtht  = function
| [] -> 0
| _ :: tl -> 1 + length tl

val lengtht : 'a list -> int = <fun>


In [48]:
lengtht [1;2;3]

- : int = 3


In [49]:
let make_list n = List.init ~f:(fun x -> x)

val make_list : 'a -> int -> int Core.List.t = <fun>


In [55]:
lengtht (make_list 10);;

error: compile_error

尾递归版本

In [52]:
let rec length_plus_n l n =
match l with
| [] -> n
| _ :: tl -> length_plus_n tl (n + 1)

val length_plus_n : 'a list -> int -> int = <fun>


In [53]:
let length l = length_plus_n l 0;;

val length : 'a list -> int = <fun>


In [54]:
length (make_list 10000000);;

error: compile_error

下面来考虑一个情况，一个函数（调用者）调用了另外一个函数（被调用者）。对于被调用者返回的值，如果调用者除了返回这个值之外不做任何其他处理，则认为这个调用是一个尾调用。尾调用优化很有意义，因为调用者完成一个尾调用时，调用者的帧栈不再使用，所以不用保留这个帧栈，因此，编译器可以服用调用者的帧栈。

## 更简洁更快速的模式

In [56]:
let rec destutter list = 
match list with
| [] -> []
| [hd] -> [hd]
| hd:: hd' :: tl ->
if hd = hd' then destutter(hd' :: tl)
else hd :: destutter(hd' :: tl)

val destutter : 'a list -> 'a list = <fun>


首先考虑效率。上面的`destutter`代码存在一个问题是，有些情况下，他会在箭头右边重新创建左边已经存在的一个值，因此模式`[hd] -> [hd]`实际上会分配一个新的列表元素，而实际上，他应该只返回匹配的列表。这里可以通过使用`as`模式减少分配，利用`as`模式，我们可以为模式或子模式匹配的部分声明一个名字。另外，我们还将使用`function`关键字，这样就不需要再使用一个显示的`match`了。

In [57]:
let rec destutter = function
| [] | [_] as l -> l
| hd :: (hd' :: _ as tl) -> 
if hd = hd' then destutter tl
else hd :: destutter tl

val destutter : 'a list -> 'a list = <fun>


In [58]:
(=)


- : 'a -> 'a -> bool = <fun>


In [59]:
(fun x -> x + 1) = (fun x -> x + 1)

error: runtime_error

多态比较确实存在一些限制，例如，如果遇到一个函数值，就会运行失败

需要说明，`when`字句有一些缺点。前面已经提到，与模式匹配相关的静态检查依赖于这样一个事实：模式在表达能力方面是有局限性的。一旦能力增强，可以为模式增加任意的条件，有些东西就会丢失。具体来讲，编译器在确定一个`match`是否完备或者某种情况是否多余时，能力会减弱。

In [60]:
let rec count_some list = 
match list with
| [] -> 0
| x :: tl when Option.is_none x -> count_some tl
| x :: tl when Option.is_some x ->  1 + count_some tl

File "[60]", line 2, characters 0-128:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
_::_
(However, some guarded clause may match this value.)


val count_some : 'a Core.Option.t list -> int = <fun>


In [61]:
count_some [Some 3; None; Some 4]

- : int = 2


In [63]:
let rec count_some list = 
match list with
| [] -> 0
| None :: tl -> count_some tl
| Some _ :: tl -> 1 + count_some tl

val count_some : 'a option list -> int = <fun>
